In [2]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as npA
from datetime import timedelta, datetime
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm
import seaborn as sns
import os
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import datetime
from sklearn.cluster import KMeans
from geopy.distance import geodesic 
import warnings
from collections import defaultdict
import requests
import numpy as np

In [6]:
# Leer el DataFrame desde el archivo CSV
df = pd.read_csv("cluster_metrics.csv")

# Función para obtener la distancia desde OSRM
def get_route_distance(lat1, lon1, lat2, lon2, osrm_url="http://router.project-osrm.org"):
    try:
        # Validación de entrada
        if not all(map(lambda x: isinstance(x, (float, int)), [lat1, lon1, lat2, lon2])):
            raise ValueError("Las coordenadas deben ser números (float o int).")
        
        # Construcción de la URL de OSRM
        route_url = f"{osrm_url}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=full&geometries=geojson"
        
        # Llamada a la API
        response = requests.get(route_url)
        
        if response.status_code == 200:
            result = response.json()
            # Extraer distancia (en metros, convertir a km)
            distance = result['routes'][0]['distance'] / 1000  # en km
            return distance
        else:
            print(f"Error en la API OSRM: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error al calcular la ruta: {e}")
        return None

# Obtener distancias y desniveles para todas las combinaciones de origen-destino
routes = []
for i, row1 in df.iterrows():
    for j, row2 in df.iterrows():
        if i != j:  # Evitar calcular rutas entre el mismo punto
            distance = get_route_distance(
                row1["Centroide_Lat"], row1["Centroide_Lon"], 
                row2["Centroide_Lat"], row2["Centroide_Lon"]
            )
            desnivel = row2["Altitud"] - row1["Altitud"]  # Calcular el desnivel
            routes.append({
                "Origen": row1["Cluster_ID"],
                "Destino": row2["Cluster_ID"],
                "Distancia_km": distance,
                "Desnivel": desnivel
            })

routes_df["Origen"] = routes_df["Origen"].astype(int)
routes_df["Destino"] = routes_df["Destino"].astype(int)
# Convertir resultados a DataFrame
routes_df = pd.DataFrame(routes)

# Mostrar el DataFrame resultante
routes_df

,Origen,Destino,Distancia_km,Desnivel
0,0.0,1.0,6.3949,-34.0
1,0.0,2.0,5.4500,-30.0
2,0.0,3.0,4.8313,1.0
3,0.0,4.0,3.8644,18.0
4,0.0,5.0,4.6489,-14.0
...,...,...,...,...
1555,39.0,34.0,6.6091,-13.0
1556,39.0,35.0,8.0229,2.0
1557,39.0,36.0,3.3870,-1.0
1558,39.0,37.0,9.0339,2.0


In [16]:
# Factor de impacto del desnivel (puedes ajustar este valor según las pruebas o especificaciones)
k = 0.1  # Incremento de Wh por metro de desnivel positivo

# Función para calcular el consumo
def calcular_consumo(distancia_km, desnivel):
    # Consumo base por distancia
    consumo = distancia_km * 13.5
    
    # Ajuste por desnivel
    if desnivel > 0:  # Subida
        consumo += k * desnivel
    elif desnivel < 0:  # Bajada
        consumo += k * desnivel  # Bajadas conservan el impacto, podría ajustarse si hay recuperación de energía
    return max(consumo, 0)  # El consumo no puede ser negativo

# Aplicar la función a cada fila del DataFrame
routes_df["Consumo_Wh"] = routes_df.apply(
    lambda row: calcular_consumo(row["Distancia_km"], row["Desnivel"]), axis=1
)
routes_df.to_csv('rutas.csv', index=False)
# Mostrar el DataFrame actualizado
routes_df


,Origen,Destino,Distancia_km,Desnivel,Consumo_Wh
0,0,1,6.3949,-34.0,82.93115
1,0,2,5.4500,-30.0,70.57500
2,0,3,4.8313,1.0,65.32255
3,0,4,3.8644,18.0,53.96940
4,0,5,4.6489,-14.0,61.36015
...,...,...,...,...,...
1555,39,34,6.6091,-13.0,87.92285
1556,39,35,8.0229,2.0,108.50915
1557,39,36,3.3870,-1.0,45.62450
1558,39,37,9.0339,2.0,122.15765
